In [1]:
import numpy as np
from pyscf import gto, scf, fci

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for spinorbitals
def ONERDMFT_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals
def ONERDMFT_Umrigar_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            if a == b:
                print()
            else:
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [5]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #
#mol.basis = "ccpvdz"
#, mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()

N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu, naturalC = np.linalg.eigh(gamma)

(2, 2, 2, 2)
converged SCF energy = -2.85516042615445


In [6]:
PE = expand_matrix(P)
CE = expand_matrix(C)
SE = expand_matrix(S)
gamma = np.matmul(np.matmul(CE.T,np.matmul(np.matmul(SE,PE),SE)), CE)
occu, naturalC = np.linalg.eigh(gamma)
naturalCTT = np.matmul(CE,naturalC)

In [7]:
print(naturalCTT)
E = ONERDMFT_Umrigar_hartree_energy(eri, naturalCTT, occu)
print(E, occu)
E = ONERDMFT_hartree_energy(eri, naturalCTT, occu)
print(E, occu)

[[ 1.14981805  0.          0.         -0.59208126]
 [-1.1869588   0.          0.         -0.51358601]
 [ 0.          1.14981805 -0.59208126  0.        ]
 [ 0.         -1.1869588  -0.51358601  0.        ]]




1.0269071688753775 [-7.56863481e-17 -7.56863481e-17  1.00000000e+00  1.00000000e+00]
2.0538143377507536 [-7.56863481e-17 -7.56863481e-17  1.00000000e+00  1.00000000e+00]
